In [1]:
from drn_interactions.config import ExperimentInfo, Config
from drn_interactions.transforms import SpikesHandler
from drn_interactions.io import load_derived_generic
from drn_interactions.stats import mannwhitneyu_plusplus
import numpy as np
import pandas as pd
from scipy.stats import zscore, zmap
# %load_ext rpy2.ipython

In [2]:
def load_drug_data(
    block="chal", 
    t_start=-600, 
    t_stop=1200, 
    bin_width=60,
    sessions=None,
    merge_neuron_types=False,
    group_names=None,
    ):
    df_binned = SpikesHandler(
        block=block,
        bin_width=bin_width,
        session_names=sessions,
        t_start=t_start,
        t_stop=t_stop,
    ).binned
    df_binned["block"] = np.where(df_binned["bin"] < 0, "pre", "post")
    neuron_types = load_derived_generic("neuron_types.csv")[["neuron_id", "neuron_type", "session_name", "group_name"]]
    df_binned = df_binned.merge(neuron_types)
    if group_names is not None:
        df_binned = df_binned.query("group_name in @group_names")
    return df_binned



In [6]:
neuron_types = load_derived_generic("neuron_types.csv")
df_chal =load_drug_data()
df_chal["zcounts"] = df_chal.groupby("neuron_id")["counts"].transform(lambda x: zmap(x, x[:60]))
df_chal = df_chal.dropna()
df_chal.to_parquet(Config.derived_data_dir / "chal_binned.parquet", index=False)

In [4]:
df_chal.head(2)

,neuron_id,bin,counts,session_name,block,neuron_type,group_name,zcounts
0,1011,-600,442,hamilton_10,pre,SR,acute_cit,-0.820422
1,1011,-540,559,hamilton_10,pre,SR,acute_cit,0.337038


In [5]:
def mwu(df):
    pre = df.query("block == 'pre'")["zcounts"].values
    post = df.query("block == 'post'")["zcounts"].values
    return mannwhitneyu_plusplus(pre, post, names=("pre", "post"))

df_responders = (
    df_chal
    .groupby("neuron_id")
    .apply(mwu)
    .reset_index()
    .merge(neuron_types)
)
df_responders.to_csv(Config.derived_data_dir / "chal_responders.csv", index=False)


# Way

In [8]:
df_way = load_drug_data(block="way")
df_way["zcounts"] = df_way.groupby("neuron_id")["counts"].transform(lambda x: zmap(x, x[:10]))
df_way = df_way.dropna()

df_responders = (
    df_way
    .groupby("neuron_id")
    .apply(mwu)
    .reset_index()
    .merge(neuron_types)
)
df_responders.to_csv(Config.derived_data_dir / "way_responders.csv", index=False)
df_way.to_parquet(Config.derived_data_dir / "way_binned.parquet", index=False)